# ECPC

This was a Crypto challenge from Day 2 of the Black Hat MEA CTF Finals, written by Polymero. The challenge file can be found [here](ecpc.py), but I will also describe a high-level overview of the challenge here.

The setup is [ECDSA](https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm) on [Curve25519](https://en.wikipedia.org/wiki/Curve25519), which has the following parameters:
- The curve $E$ is $y^2 = x^3+486662x^2+x$ on the field $\textrm{GF}(p)$ with $p=2^{255}-19$.
- The base point $G$ has $G_x=9$. This generates a cyclic subgroup of prime order $n \approx \frac{p}{8}$.
- A private key $d$ is randomly generated, but the public key $Q=dG$ is never revealed to us.
- The hash function is $h(m) = \textrm{sha256}(Q\|m)$, which depends on the unknown parameter $Q$.
- To sign a message $m$, a random $k$ is generated and the signature is $(r,s) = \left( (kG)_x \bmod{n}, k^{-1}(h+rd) \bmod{n} \right)$.

Then the challenge is basically as follows:
- We are given the hash of the empty string, i.e. $h_0 := \textrm{sha256}(Q)$. This is called the Connection ID. We are not given $Q$ itself.
- The flag is represented as a stream of bits, and then for each bit:
    - If the bit is set, we are given a random signature $(r, s)$ of the message $m=1$.
    - If the bit is unset, we are given a random element of $\mathbb{Z}_n^2$.
- We have a signature oracle which can spit out a valid signature $(r, s)$ for any message $m$ that we give it.
- The job is to recover the flag, i.e. by distinguishing the valid signatures of $m=1$ from the random pairs.

Or at least, that was the original challenge, which we will call Level 1. Part of the purpose of this write-up is to keep adding levels to make it increasingly difficult but still solvable.

We begin by importing the usual modules, and just defining a bunch of useful helper functions.

In [1]:
from pwn import *
from sage.all import *
from ast import literal_eval
from Crypto.Util.number import long_to_bytes, bytes_to_long
from hashlib import sha256
import base64
context.log_level = 'error'

p = 2**255 - 19
E = EllipticCurve(GF(p), [0, 486662, 0, 1, 0])
n = 7237005577332262213973186563042994240857116359379907606001950938285454250989
G = -E.lift_x(Integer(9))

def create_process():
    return process(['python', 'ecpc.py'])

def get_connection_id(sh):
    sh.readuntil(b'ID = ')
    return int(sh.readline(False))

# In the original challenge, the (r, s) pairs are given in base64
def get_rs_pairs(sh):
    sh.readuntil(b'Flag = ')
    line = sh.readline(False)
    assert len(line) % 43 == 0
    values = [bytes_to_long(base64.urlsafe_b64decode(line[i:i+43]+b'=')) for i in range(0, len(line), 43)]
    return [(values[i], values[i+1]) for i in range(0, len(values), 2)]

def get_signature(sh, data):
    sh.sendline(data.hex().encode())
    sh.readuntil(b's) = ')
    return literal_eval(sh.readline(False).decode())

# In the original challenge, the point Q is encoded as the following string
def get_hash(Q, m=b''):
    z = f'Point ({Q[0]}, {Q[1]}) on Curve 25519'
    return bytes_to_long(sha256(z.encode() + m).digest())

## Level 1: The original challenge

This is basically a warm-up exercise. Since we can sign the empty string, for which we already know the hash, this gives us a signature

$$(r,s) = \left( (kG)_x \bmod{n}, k^{-1}(h_0+rd) \bmod{n} \right),$$

and we can rearrange this to get

$$\left( s^{-1}(h_0 G + r Q)\right)_x \bmod{n} = r.$$

Or equivalently,

$$h_0 G + r Q = sP \textrm{ for some $P$ satisfying $P_x \bmod{n} = r$.}$$

We already know $h_0$, $G$, $r$, and $s$. We don't know $P$ outright, but it turns out there are really only up to 16 possibilities for P:

$$P \in \textrm{lifts}(r) := \left\{ P \in E : P_x \bmod{n} = r \right\}$$

which we can enumerate by lifting all possible values of $x$ which are congruent to $r \pmod{n}$.

Essentially we are done, since we can just test all possible $Q = r^{-1} (s P - h_0 G)$ until we find the one that matches our Connection ID. Once we have $Q$ we can generate the hash of any message, in particular the one corresponding to $m=1$, so that it becomes trivial to check for a valid signature.

In [2]:
def f(P):
    return int(P[0]) % n

def lifts(r):
    return [P for x in range(r, p, n) for P in E.lift_x(Integer(x), True)]

In [3]:
with create_process() as sh:
    h0 = get_connection_id(sh)
    rs_pairs = get_rs_pairs(sh)
    r, s = get_signature(sh, b'')

Qs = [(s * P - h0 * G) * pow(r, -1, n) for P in lifts(r)]
Q = next(Q for Q in Qs if get_hash(Q) == h0)
print(f'{Q = }')
h1 = get_hash(Q, b'1')
result = unbits(f((h1 * G + r * Q) * pow(s, -1, n)) == r for r, s in rs_pairs)
print(result)

Q = (41119926527203953096141857241850528609570113026221016510499019193864343849405 : 21528923670270134553100840596428476217908560325371257203457139871004876615706 : 1)
b'flag{spl4t_th3m_bugs}'


## Level 2: Level 1 + No empty input

It turns out that the above solution was unintended, and that we were not meant to be able to submit the empty message. Oh well, we can still use a [length extension attack](https://en.wikipedia.org/wiki/Length_extension_attack) on SHA-256 to obtain a known hash.

What this means is that we can craft a special message $M$ such that we can compute $\textrm{sha256}(Q\|M)$ only from the value of $\textrm{sha256}(Q)$ and the length of $Q$. Roughly speaking, this message $M$ is just the padding of the original message. From local testing, we note that with high probability, the string representation of $Q$ will have length 179 or 178 (or less commonly 177). For simplicity we just assume it has length 179, and simply retry if it fails.

Once we have this known hash $h' = h(M)$, we can compute $Q$ as in Level 1 and proceed similarly.

In [4]:
def sha256extend(prev_hash, length):
    assert len(prev_hash) == 32
    hs = [int.from_bytes(prev_hash[i:i+4], 'big') for i in range(0, 32, 4)]
    extra_block = b'\x80' + bytes(55) + ((8 * length + 576) & -512).to_bytes(8, 'big')

    k = [int(p**(1/3)%1*2**32) for p in range(2,312) if not [1 for x in range(2,p) if p%x==0]]
    w = [int.from_bytes(extra_block[i:i+4], 'big') for i in range(0, len(extra_block), 4)]

    rrot = lambda x, n: x >> n | x << (32 - n)
    for i in range(48):
        s0 = rrot(w[i + 1], 7) ^ rrot(w[i + 1], 18) ^ (w[i + 1] >> 3)
        s1 = rrot(w[i + 14], 17) ^ rrot(w[i + 14], 19) ^ (w[i + 14] >> 10)
        w.append((w[i] + w[i+9] + s0 + s1) % 2**32)

    a,b,c,d,e,f,g,h=hs
    for i in range(64):
        s0 = rrot(a, 2) ^ rrot(a, 13) ^ rrot(a, 22)
        maj = (a & b) ^ (a & c) ^ (b & c)
        t2 = s0 + maj
        s1 = rrot(e, 6) ^ rrot(e, 11) ^ rrot(e, 25)
        ch = (e & f) ^ (~e & g)
        t1 = h + s1 + ch + k[i] + w[i]
        a,b,c,d,e,f,g,h = (t1+t2)%2**32,a,b,c,(d+t1)%2**32,e,f,g

    hs = [(x+y)%2**32 for x,y in zip(hs, [a,b,c,d,e,f,g,h])]
    return b'\x80' + bytes((55 - length) % 64) + (8 * length).to_bytes(8, 'big'), b''.join(x.to_bytes(4, 'big') for x in hs)

In [5]:
with create_process() as sh:
    h0 = get_connection_id(sh)
    ext = sha256extend(long_to_bytes(h0), 179) # might be < 179, rerun if fail
    m, h0_ext = ext[0], bytes_to_long(ext[1])
    rs_pairs = get_rs_pairs(sh)
    r, s = get_signature(sh, m)

Qs = [(s * P - h0_ext * G) * pow(r, -1, n) for P in lifts(r)]
Q = next(Q for Q in Qs if get_hash(Q) == h0)
print(f'{Q = }')
h1 = get_hash(Q, b'1')
result = unbits(f((h1 * G + r * Q) * pow(s, -1, n)) == r for r, s in rs_pairs)
print(result)

Q = (26069349793998640802015468700598441466386357307181202119782485962808041289951 : 28004835041040223710951188044377440044233863632000018265844173162216422297837 : 1)
b'flag{spl4t_th3m_bugs}'


## Level 3: Level 2 + No input at all

Upon further reflection, why should we need to know $h$ at all to solve for $Q$? The only hash we are really interested in at the end is $h_1$ (i.e. hash of `b'1'`), and even then we are content to just know the value of $h_1 G$, which we will just call $H$.

What this means is that we will use the given $(r, s)$ pairs, since we know that many of them have been hashed against $h_1$. Our equation from earlier then simply becomes

$$H + rQ = sP.$$

And if we have two such pairs $(r_1, s_1)$ and $(r_2, s_2)$, then since $H$ is fixed, we can solve for $Q$ by subtracting the two equations:

$$Q = \frac{s_1 P_1 - s_2 P_2}{r_1 - r_2}.$$

This gives us up to 256 different values of $Q$, but as before we can just test for the correct one by comparing against the hash.

What we do need to know then is the position of the 1 bits, which we know from the structure of the flag, i.e. it begins with either 'F' or 'f' (so the bits in positions 1, 5, and 6 are set).

In [6]:
with create_process() as sh:
    h0 = get_connection_id(sh)
    rs_pairs = get_rs_pairs(sh)

r1, s1 = rs_pairs[1]
r2, s2 = rs_pairs[5]

Qs = [(s1 * P1 - s2 * P2) * pow(r1 - r2, -1, n) for P1 in lifts(r1) for P2 in lifts(r2)]
Q = next(Q for Q in Qs if get_hash(Q) == h0)
print(f'{Q = }')
h1 = get_hash(Q, b'1')
result = unbits(f((h1 * G + r * Q) * pow(s, -1, n)) == r for r, s in rs_pairs)
print(result)

Q = (26433301926690947870549036439467418456044102195167929779743835436557044585841 : 1048587294329967287535507047458533751328655839839745104360659757795803841561 : 1)
b'flag{spl4t_th3m_bugs}'


## Level 4: Level 3 + No connection ID

This is actually not significantly harder. Instead of comparing against the hash $h_0$, we can just check whether $Q$ satisfies

$$\left( s^{-1} \left(h(Q\|1)G+rQ \right) \right)_x \bmod{n} = r$$

for a valid signature pair $(r, s)$.

In [7]:
with create_process() as sh:
    rs_pairs = get_rs_pairs(sh)
    
r1, s1 = rs_pairs[1]
r2, s2 = rs_pairs[5]

Qs = [(s1 * P1 - s2 * P2) * pow(r1 - r2, -1, n) for P1 in lifts(r1) for P2 in lifts(r2)]
Q = next(Q for Q in Qs if f((get_hash(Q, b'1') * G + r1 * Q) * pow(s1, -1, n)) == r1)
print(f'{Q = }')
h1 = get_hash(Q, b'1')
result = unbits(f((h1 * G + r * Q) * pow(s, -1, n)) == r for r, s in rs_pairs)
print(result)

Q = (10643219452687178986054127809507943543171709038278342124630545704379849158439 : 28775439047321395735000262278988580175502235090493808426765938559191385486561 : 1)
b'flag{spl4t_th3m_bugs}'


## Level 5: Level 4 + No hash information

One common pattern in Levels 1-4 is that we have used the hash in some way, to either verify $Q$ or to construct $h_1$ from $Q$. But what if the hash was replaced with a 100% black box?

We can use the same pair of simultaneous equations as before, but instead of solving for just $Q$, we solve for the pair $(H, Q)$ and then test it against a third point. In this case we actually have two possible solutions, since $(-H, -Q)$ also satisfies the equation. We have no way of knowing which pair is the "correct" one, but it doesn't matter since they are equivalent for the purpose of signature verification.

In [8]:
with create_process() as sh:
    rs_pairs = get_rs_pairs(sh)
    
r1, s1 = rs_pairs[1]
r2, s2 = rs_pairs[5]
r3, s3 = rs_pairs[6]

def make_HQ_pairs(P, Q, r, s):
    H = s * P - r * Q
    return H, Q

HQs = [make_HQ_pairs(P1, (s1 * P1 - s2 * P2) * pow(r1 - r2, -1, n), r1, s1) for P1 in lifts(r1) for P2 in lifts(r2)]
H, Q = next((H, Q) for H, Q in HQs if f((H + r3 *Q) * pow(s3, -1, n)) == r3)
print(f'{Q = }')
print(f'{H = }')
result = unbits(f((H + r * Q) * pow(s, -1, n)) == r for r, s in rs_pairs)
print(result)

Q = (19198444554718211454190171853651619997653418802152490436973033663284823199962 : 2897881308436666184363894451494927572239787798192449402402656921360772471327 : 1)
H = (40451718790325014625516898333866677362697118181613017113182071195725798237555 : 7261194096238398785555382782844097659873664766801798753810175272918065773485 : 1)
b'flag{spl4t_th3m_bugs}'


## Level 6: Level 5 + No $s$ provided

Surprise! This turns out to be the easiest of the lot. With no $s$ provided, we cannot actually verify if a signature is valid, and are reduced to distinguishing whether a given $r$ is the possible x-coordinate (mod $n$) of some $kG$, or if it's merely random.

Some quick handwavy maths shows that roughly half of all values of $x$ can be lifted to $E$. But we have a cofactor of 8 here which means that our given value of $r$ can be unprojected to 8 values of $x$, which you would think gives it a $\frac{1}{256}$ chance of getting identified as a random value.

However, the cofactor is a double-edged sword here, as the full curve $E$ actually has $8n$ points, so there's only a $\frac{1}{8}$ probability that a given $x$ is in fact on our order-$n$ subgroup. This means that the set of all possible x-coordinates of $kG$ only covers roughly $\frac{1}{16}$ of $\textrm{GF}(p)$.

This also means that the probability that a random $r$ can be identified as being random is $\left(1-\frac{1}{16}\right)^8 \approx 59.7\%$, and you can test this by simply lifting all possible $x$ values and seeing if any of them satisfy $nP=0$.

This gives us a probabilistic way to recover out flag:
- Start with the bitstream consisting of all 1s.
- Run the process, which will turn roughly 60\% of the actual 0s into a 0 on our side.
- Run a second time, so that now roughly 84\% of the actual 0s have become a 0 on our side.
- If we run it 10 times, this increases to 99.99\%.
- If we run it 20 times, this increases to 99.999999\%.

So depending on your probabilistic tolerance, running 10 times in pretty good for a regular-sized flag, but 20 times should cover just about every case.

In [9]:
result = -1
for _ in range(10): # for a longer flag maybe increase it to 20
    
    with create_process() as sh:
        rs = [r for r,s in get_rs_pairs(sh)]
    
    valids = [any(n * P == 0 for P in lifts(r)) for r in rs]
    result &= bytes_to_long(unbits(valids))
    print(long_to_bytes(result))

b'f\xfc\xfd\xe7\x7f\xff\xf6n5\xf7\xffuy{\xef\xdf\xf3}o{}'
b'f|qg{\xf3tl5w_uhsm\xdf\xe2}gs}'
b'flag{\xf3pl4v_thsm\xdfbugs}'
b'flag{spl4t_th3m\xdfbugs}'
b'flag{spl4t_th3m_bugs}'
b'flag{spl4t_th3m_bugs}'
b'flag{spl4t_th3m_bugs}'
b'flag{spl4t_th3m_bugs}'
b'flag{spl4t_th3m_bugs}'
b'flag{spl4t_th3m_bugs}'
